In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as ss
import os
import yaml, inspect, os, sys, subprocess, pprint, shutil, argparse

import matplotlib.pyplot as plt
import re
from collections import defaultdict
import copy




In [2]:
def get_yaml_object(yaml_filename):
    yaml_object = {}
    with open(yaml_filename) as stream:
        try:
            yaml_object = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return yaml_object

def write_yaml_to_file(yaml_filename, yaml_object):
    with open(yaml_filename, 'w') as outfile:
        yaml.dump(yaml_object, outfile, default_flow_style=False)

In [3]:
def modify_distributions(layer_yaml, input_sparsity = None, weight_sparsity = None):
    new_layer_yaml = copy.deepcopy(layer_yaml)
    if input_sparsity is not None:
        new_layer_yaml['problem']['instance']['densities']['Inputs']['density'] = input_sparsity
    if weight_sparsity is not None:
        new_layer_yaml['problem']['instance']['densities']['Weights']['density'] = weight_sparsity
    return new_layer_yaml

In [4]:
def modify_layer_with_sparsities(layer_yaml, input_sparsities):
    return [
        (modify_distributions(layer_yaml,input_sparsity), input_sparsity) for input_sparsity in input_sparsities
    ]

In [5]:
def write_modified_layers(original_layer_name, modified_layers_with_labels):
    filename_without_yaml = original_layer_name.strip('.yaml')
    for modified_layer, label in modified_layers_with_labels:
        new_filename = filename_without_yaml + '_sparse=' + str(label) + '.yaml'
        print(f'writing {label} to {new_filename}')
        write_yaml_to_file(new_filename,modified_layer)

In [6]:
yaml_filename = os.path.join(
    os.getcwd(),
    '../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform_experiment/L07.yaml'
)
layer_yaml = get_yaml_object(yaml_filename)
# print(layer_yaml)
yaml.dump(layer_yaml, sys.stdout)



problem:
  instance:
    C: 64
    E: 32
    F: 32
    G: 1
    M: 64
    N: 1
    R: 1
    S: 1
    densities:
      Inputs:
        density: 0.73
        distribution: hypergeometric
      Weights:
        density: 0.52
        distribution: hypergeometric
  shape:
    coefficients:
    - default: 1
      name: Wstride
    - default: 1
      name: Hstride
    - default: 1
      name: Wdilation
    - default: 1
      name: Hdilation
    data-spaces:
    - name: Weights
      projection:
      - - - C
      - - - M
      - - - G
      - - - R
      - - - S
    - name: Inputs
      projection:
      - - - N
      - - - C
      - - - G
      - - - R
          - Wdilation
        - - E
          - Wstride
      - - - S
          - Hdilation
        - - F
          - Hstride
    - name: Outputs
      projection:
      - - - N
      - - - G
      - - - M
      - - - F
      - - - E
      read-write: true
    dimensions:
    - R
    - S
    - E
    - F
    - C
    - M
    - N
    - G
    nam

In [7]:
input_sparsities = [i/10 for i in range(1,10)]
write_modified_layers(
    yaml_filename,
    modify_layer_with_sparsities(
        layer_yaml,
        input_sparsities
    )
)

writing 0.1 to /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform_experiment/L07_sparse=0.1.yaml
writing 0.2 to /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform_experiment/L07_sparse=0.2.yaml
writing 0.3 to /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform_experiment/L07_sparse=0.3.yaml
writing 0.4 to /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform_experiment/L07_sparse=0.4.yaml
writing 0.5 to /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform_experiment/L07_sparse=0.5.yaml
writing 0.6 to /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sp